## Binary Classification - Titanic: Machine Learning from Disaster

https://www.kaggle.com/c/titanic

- 유명한 자료인 타이타닉 데이터 셋으로 생존과 사망에 대한 분류 문제를 풀어보도록 하겠습니다.

![대체 텍스트](../figures2/binary.png)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt 

random.seed(1215)
torch.manual_seed(1215)

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load Data

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('{name} Uploaded.'.format(name=fn))

In [ ]:
import pandas as pd

train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./test.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/titanic/train.csv'

In [ ]:
train_data.head(10)

In [ ]:
test_data.head(10)

# Preprocessing
- Pclass, Sex, Age, SibSp, Parch, Fare 총 6개의 feature를 사용하겠습니다.
- 별도의 feature engineering은 하지 않겠습니다.
- kaggle data이기 때문에 test_data의 "Survived" 열은 없습니다.
- 우리가 예측한 결과를 csv 파일로 제출하는 형태입니다.

### categorical variable 처리

In [ ]:
train_data["Sex"].head(10)

In [ ]:
train_data["Sex"] = train_data["Sex"].map({"male": 1, "female": 0})
test_data["Sex"] = test_data["Sex"].map({"male": 1, "female": 0})

In [ ]:
train_data["Sex"].head(10)

### 데이터셋 구성

In [ ]:
data_X = train_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]]
data_y = train_data["Survived"]

In [ ]:
len(data_X)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(data_X, data_y, test_size=0.3)

In [ ]:
len(train_X), len(data_X)*.7

In [ ]:
len(test_X), len(data_X)*.3

### null 값 확인 (결측값)

In [ ]:
print('Number of null in train_X:', train_X.isnull().sum())
print("\n",'Number of null in test_X:', test_X.isnull().sum())
print("\n",'Number of nullin train y:', train_y.isnull().sum())
print("\n",'Number of nullin train y:', test_y.isnull().sum())

print("\n",np.mean(train_X["Age"]))

- 간단하게 age와 fare의 na 값은 평균으로 대체

In [ ]:
pd.options.mode.chained_assignment = None

train_X.loc[:,"Age"] = train_X.loc[:,"Age"].replace(np.nan,30)
test_X.loc[:,"Age"] = test_X.loc[:,"Age"].replace(np.nan,30)

In [ ]:
print('Number of null:', train_X.isnull().sum())
print("\n",'Number of null:', test_X.isnull().sum())

In [ ]:
train_y.head()

In [ ]:
len(train_X), len(test_X), len(train_y), len(test_y)

# Train model (PyTorch)
1. dataset, dataloader 정의
2. 모델 정의
3. 모델 학습 
4. 모델 평가

- train, test dataset을 출력할 수 있는 class를 만듭니다.

In [ ]:
class simple_dataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
type(test_X)

In [ ]:
# 정규화합니다
scaler = StandardScaler()
train_data = simple_dataset(torch.FloatTensor(scaler.fit_transform(train_X.to_numpy())), torch.LongTensor(train_y.to_numpy()))
test_data = simple_dataset(torch.FloatTensor(scaler.fit_transform(test_X.to_numpy())), torch.LongTensor(test_y.to_numpy()))

In [ ]:
len(train_data), len(test_data)

In [ ]:
train_data.__getitem__(0)

In [ ]:
train_data[0]

## Modeling

In [ ]:
class Binary_Classification(nn.Module):
    
    def __init__(self, num_features, num_classes):
        super(Binary_Classification, self).__init__()
        
        self.Layer_1 = nn.Linear(num_features, 30)
        
        ################################################
        #  TODO    Layer 만들기                        #
        ################################################
        
        ################################################
        #  가능한 답은 하나가 아니므로 성능이 더 좋을  #
        #  법할 모델을 다양하게 시도해보시길 바랍니다  #
        ################################################

        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        
        x = self.Layer_1(inputs)
        x = self.relu(x)
        
        ################################################
        #  TODO       feature를 layer를 통과시키기     #
        ################################################

        ################################################
        #  가능한 답은 하나가 아니므로 성능이 더 좋을  #
        #  법할 모델을 다양하게 시도해보시길 바랍니다  #
        ################################################
        
        x = self.Layer_out(x)

        return x

## Training

In [ ]:
EPOCHS = 3000
BATCH_SIZE = 891

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

model = Binary_Classification(num_features=6, num_classes=2)

criterion = nn.CrossEntropyLoss() # https://pytorch.org/docs/stable/nn.html
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 데이터 사이즈 확인 

In [ ]:
for epoch in range(EPOCHS):
    for X_batch, y_batch in train_loader:
        print(X_batch.size(), y_batch.size())
        break
    break

In [ ]:
loss_list = []
acc_list = []
for epoch in range(EPOCHS):
    for i, (X_batch, y_batch) in enumerate(train_loader):
        
        #Forward 
        y_output = model(X_batch)
        loss = criterion(y_output, y_batch) #CELoss: The input is expected to contain raw, unnormalized scores for each class.
        
        #Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #misc (acc 계산, etc) 
        y_pred = torch.max(y_output, 1)[1]
        acc = accuracy_score(y_pred.data.cpu(), y_batch.data.cpu())
        loss_list.append(loss.item())
        acc_list.append(acc)

    if (epoch+1) % 30 == 0:
        print('Epoch [{}/{}] Step [{}/{}] Loss: [{:.4f}] Train ACC [{:.2f}%]'.format(epoch+1, EPOCHS, \
                                                                                   i+1, len(train_loader), loss.item(), acc*100))

In [ ]:
plt.plot(loss_list)

In [ ]:
plt.plot(acc_list)

## Test

In [ ]:
test_y_pred = []
test_acc_list = []
with torch.no_grad():

    for X_batch, y_batch in test_loader:    
        #Forward
        y_output = model(X_batch)
        
        #misc (acc 계산, etc) 
        y_pred = torch.max(y_output, 1)[1]
        test_y_pred.append(y_pred) ##
        
        acc = accuracy_score(y_pred.data.cpu(), y_batch.data.cpu())
        test_acc_list.append(acc)
    test_acc = np.mean(test_acc_list)
print('Test ACC: [{:.2f}%]'.format(test_acc*100))

In [ ]:
len(test_y_pred), test_y_pred[:10]

# *Alternative code*

In [ ]:
class Binary_Classification_layer(nn.Module):
    
    def __init__(self, num_features, num_classes):
        super(Binary_Classification_layer, self).__init__()
        
        self.Layer_1 = nn.Linear(num_feature, num_classes)
                     
        
        ################################################
        #       TODO                                   #
        ################################################

        ################################################
        #  가능한 답은 하나가 아니므로 성능이 더 좋을  #
        #  법할 모델을 다양하게 시도해보시길 바랍니다  #
        ################################################


    def forward(self, inputs):
        
        x = self.Layer_1(inputs)      
        
        ################################################
        #       TODO                                   #
        ################################################    
        
        ################################################
        #  가능한 답은 하나가 아니므로 성능이 더 좋을  #
        #  법할 모델을 다양하게 시도해보시길 바랍니다  #
        ################################################


        return x

# *Assignment*

- 본인만의 모델을 만들어 Test ACC 79%를 달성해보세요!

In [ ]:
EPOCHS = 1000
BATCH_SIZE = 200

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

model = Binary_Classification_layer(num_features=6, num_classes=2)

criterion = nn.CrossEntropyLoss() # https://pytorch.org/docs/stable/nn.html
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss_list = []
train_acc_list = []

test_acc_list = []

for epoch in range(EPOCHS):
    # train
    for 
        ################################################
        #       TODO                                   #
        ################################################ 
        X_batch = 
        y_batch = 
        
        #Forward 
        y_output = 
        loss =  #CELoss: The input is expected to contain raw, unnormalized scores for each class.
        
        #Backward
        optimizer
        loss
        optimizer
        
        #misc (acc 계산, etc) 
        y_pred = 
        train_acc =
        
        train_loss_list
        train_acc_list

        ################################################
        #  가능한 답은 하나가 아니므로 성능이 더 좋을  #
        #  법할 모델을 다양하게 시도해보시길 바랍니다  #
        ################################################

    
        
    # test
    with torch.no_grad():
        ################################################
        #       TODO                                   #
        ################################################ 
        #Forward

        
        #misc (acc 계산, etc) 

        ################################################
        #  가능한 답은 하나가 아니므로 성능이 더 좋을  #
        #  법할 모델을 다양하게 시도해보시길 바랍니다  #
        ################################################

    
    if (epoch+1) % 30 == 0:
        print('Epoch [{}/{}] Step [{}/{}] Loss: [{:.4f}] Train avg ACC [{:.2f}%] Test ACC [{:.2f}%]'.format(epoch+1, EPOCHS, \
                                                                                   i+1, len(train_loader), np.mean(train_loss_list), \
                                                                                                        np.mean(train_acc_list)*100, np.mean(test_acc_list)*100))